<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

This notebook should be run in a Watson Studio project, using **Default Spark Python** runtime environment. **If you are viewing this in Watson Studio and do not see Python 3.7 in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following Cloud services:
  * Watson OpenScale
  * Watson Machine Learning V2 plans
  * Cloud Object Storage
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio and continuous learning services. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

### Contents

- [Setup](#setup)
- [Model building and deployment](#model)
- [OpenScale configuration](#openscale)
- [Quality monitor and feedback logging](#quality)
- [Fairness, drift monitoring and explanations](#fairness)
- [Custom monitors and metrics](#custom)
- [Historical data](#historical)

### Note: These samples are using most latest openscale V2 client from public pypi. It does not cover followings aspects for now:

- Historical payload logging 
- Historical manual labeling

# Setup <a name="setup"></a>

## Spark check

In [1]:
!pip install pyspark==2.4.0 --no-cache | tail -n 1

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise 

## Package installation

In [127]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
!rm -rf /home/spark/shared/user-libs/python3.7*

!pip install --upgrade pandas==0.25.3 --no-cache | tail -n 1
!pip install --upgrade requests==2.23 --no-cache | tail -n 1
!pip install --upgrade numpy==1.20.3 --user --no-cache | tail -n 1
!pip install SciPy --no-cache | tail -n 1
!pip install lime --no-cache | tail -n 1

!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: 

## Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [61]:
CLOUD_API_KEY = "******"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key, generate an IAM token using that key and paste it below.

### WML credentials example with API key

In [22]:
WML_CREDENTIALS = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey": "******"
}

###  WML credentials example using IAM_token 

**NOTE**: If IAM_TOKEN is used for authentication and you receive unauthorized/expired token error at any steps, please create a new token and reinitiate clients authentication.

In [ ]:
 #uncomment this cell if want to use IAM_TOKEN
# import requests
# def generate_access_token():
#     headers={}
#     headers["Content-Type"] = "application/x-www-form-urlencoded"
#     headers["Accept"] = "application/json"
#     auth = HTTPBasicAuth("bx", "bx")
#     data = {
#         "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
#         "apikey": CLOUD_API_KEY
#     }
#     response = requests.post(IAM_URL, data=data, headers=headers, auth=auth)
#     json_data = response.json()
#     iam_access_token = json_data['access_token']
#     return iam_access_token

In [ ]:
#uncomment this cell if want to use IAM_TOKEN
# IAM_TOKEN = generate_access_token()
# WML_CREDENTIALS = {
#                "url": "https://us-south.ml.cloud.ibm.com",
#                "token": IAM_TOKEN
#             }

### Cloud object storage details

In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit [getting started with COS tutorial](https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-getting-started). 
You can find `COS_API_KEY_ID` and `COS_RESOURCE_CRN` variables in **_Service Credentials_** in menu of your COS instance. Used COS Service Credentials must be created with _Role_ parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription.  
`COS_ENDPOINT` variable can be found in **_Endpoint_** field of the menu.

In [23]:
COS_API_KEY_ID = "******"
COS_RESOURCE_CRN = "******" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "https://******" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [8]:
BUCKET_NAME = "******" #example: "credit-risk-training-data"

This tutorial can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you do not have a paid Cloud account or would prefer not to provision this paid service, you may use the free internal PostgreSQL service with OpenScale. Do not update the cell below.

To provision a new instance of Db2 Warehouse, locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Db2 Warehouse credentials into the cell below.

To provision a new instance of Databases for PostgreSQL, locate [Databases for PostgreSQL in the Cloud catalog](https://cloud.ibm.com/catalog/services/databases-for-postgresql), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Databases for PostgreSQL credentials into the cell below.

In [9]:
DB_CREDENTIALS = None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [10]:
KEEP_MY_INTERNAL_POSTGRES = True

## Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Model building and deployment <a name="model"></a>

In this section you will learn how to train Spark MLLib model and next deploy it as web-service using Watson Machine Learning service.

## Load the training data from github

In [11]:
from IPython.utils import io

with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv -O german_credit_data_biased_training.csv
!ls -lh german_credit_data_biased_training.csv

-rw-rw---- 1 wsuser watsonstudio 674K Aug 11 13:57 german_credit_data_biased_training.csv


In [24]:
from pyspark.sql import SparkSession
import pandas as pd
import json
import datetime

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
training_data_file_name = "german_credit_data_biased_training.csv"
df_data.head()

Row(CheckingStatus='0_to_200', LoanDuration=31, CreditHistory='credits_paid_to_date', LoanPurpose='other', LoanAmount=1889, ExistingSavings='100_to_500', EmploymentDuration='less_1', InstallmentPercent=3, Sex='female', OthersOnLoan='none', CurrentResidenceDuration=3, OwnsProperty='savings_insurance', Age=32, InstallmentPlans='none', Housing='own', ExistingCreditsCount=1, Job='skilled', Dependents=1, Telephone='none', ForeignWorker='yes', Risk='No Risk')

## Explore data

In [13]:
df_data.printSchema()

root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)



In [14]:
print("Number of records: " + str(df_data.count()))

Number of records: 5000


## Save training data to Cloud Object Storage

In [25]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [26]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

## Create a model

In [27]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Spark German Risk Model - Final"
DEPLOYMENT_NAME = "Spark German Risk Deployment - Final"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

Number of records for training: 4016
Number of records for evaluation: 984


The code below creates a Random Forest Classifier with Spark, setting up string indexers for the categorical features and the label column. Finally, this notebook creates a pipeline including the indexers and the model, and does an initial Area Under ROC evaluation of the model.

In [28]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml.feature import SQLTransformer

features = [x for x in spark_df.columns if x != 'Risk']
categorical_features = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']
categorical_num_features = [x + '_IX' for x in categorical_features]
si_list = [StringIndexer(inputCol=x, outputCol=y) for x, y in zip(categorical_features, categorical_num_features)]
va_features = VectorAssembler(inputCols=categorical_num_features + [x for x in features if x not in categorical_features], outputCol="features")

In [29]:
si_label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_label.labels)

In [30]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features")
feature_filter = SQLTransformer(statement="SELECT * FROM __THIS__")
pipeline = Pipeline(stages= si_list + [si_label, va_features, classifier, label_converter, feature_filter])
model = pipeline.fit(train_data)

**Note:** If you want filter features from model output please replace **`*`** with feature names to be retained in **`SQLTransformer`** statement.

In [31]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.703587


## Publish the model

In this section, the notebook uses the supplied Watson Machine Learning credentials to save the model (including the pipeline) to the WML instance. Previous versions of the model are removed so that the notebook can be run again, resetting all data for another demo.

In [32]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.105'

### Listing all the available spaces

In [33]:
wml_client.spaces.list(limit=10)

------------------------------------  -----------------------------------  ------------------------
ID                                    NAME                                 CREATED
cc293f72-5440-478f-a9cc-6c137784f747  sda-1-22-2021                        2021-01-28T20:11:52.608Z
5c966fdf-cb72-4fe2-9333-1a4172b714a6  scottda-deployment-space-10-29-2020  2020-10-29T15:39:35.902Z
c1e6a643-aca5-4abe-9a85-f58457b49ee9  scottda-deployment-space-9-29-2020   2020-09-29T12:39:43.524Z
------------------------------------  -----------------------------------  ------------------------


In [34]:
WML_SPACE_ID='******' # use space id here
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

### Remove existing model and deployment

In [35]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

--  ----  -------  ----
ID  NAME  CREATED  TYPE
--  ----  -------  ----


In [36]:
training_data_references = [
                {
                    "id": "product line",
                    "type": "s3",
                    "connection": {
                        "access_key_id": COS_API_KEY_ID,
                        "endpoint_url": COS_ENDPOINT,
                        "resource_instance_id":COS_RESOURCE_CRN
                    },
                    "location": {
                        "bucket": BUCKET_NAME,
                        "path": training_data_file_name,
                    }
                }
            ]

In [37]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_2.4")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
        #wml_client._models.ConfigurationMetaNames.SPACE_UID: WML_SPACE_ID,
        wml_client._models.ConfigurationMetaNames.TYPE: "mllib_2.4",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "Risk",
    }

Software Specification ID: 390d21f8-e58b-4fac-9c55-d7ceda621326


In [38]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props=model_props, 
    training_data=train_data, 
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_uid(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...
Done
Model ID: a12ed743-c537-436f-ab3c-fef1554b88d8


## Deploy the model

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [39]:
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_uid(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))



#######################################################################################

Synchronous deployment creation for uid: 'a12ed743-c537-436f-ab3c-fef1554b88d8' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='62f113b9-7583-42a8-aa5c-b1754959fe83'
------------------------------------------------------------------------------------------------


Scoring URL:https://us-south.ml.cloud.ibm.com/ml/v4/deployments/62f113b9-7583-42a8-aa5c-b1754959fe83/predictions
Model id: a12ed743-c537-436f-ab3c-fef1554b88d8
Deployment id: 62f113b9-7583-42a8-aa5c-b1754959fe83


## Sample scoring

In [40]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
        ]

scoring_payload = {"input_data": [{"fields": fields, "values": values}]}

In [41]:
scoring_response = wml_client.deployments.score(deployment_uid, scoring_payload)
scoring_response

{'predictions': [{'fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'Job',
    'Dependents',
    'Telephone',
    'ForeignWorker',
    'CheckingStatus_IX',
    'CreditHistory_IX',
    'LoanPurpose_IX',
    'ExistingSavings_IX',
    'EmploymentDuration_IX',
    'Sex_IX',
    'OthersOnLoan_IX',
    'OwnsProperty_IX',
    'InstallmentPlans_IX',
    'Housing_IX',
    'Job_IX',
    'Telephone_IX',
    'ForeignWorker_IX',
    'features',
    'rawPrediction',
    'probability',
    'prediction',
    'predictedLabel'],
   'values': [['no_checking',
     13,
     'credits_paid_to_date',
     'car_new',
     1343,
     '100_to_500',
     '1_to_4',
     2,
     'female',
     'none',
     3,
     'savings

# Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [62]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment this line if using IAM token to authenticate
wos_client = APIClient(authenticator=authenticator)
wos_client.version

'3.0.7'

## Create schema and datamart

### Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were **not** supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there **unless** there is an existing datamart **and** the **KEEP_MY_INTERNAL_POSTGRES** variable is set to **True**. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the German Credit model will be removed from OpenScale monitoring.

In [63]:
wos_client.data_marts.show()

WOS Data Mart,Data Mart created by WOS tutorial notebook,True,active,2021-07-27 18:41:18.400000+00:00,a08b0e60-6844-4c14-81cb-a3a6c9d199f9


In [64]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart a08b0e60-6844-4c14-81cb-a3a6c9d199f9


### Remove existing service provider connected with used  WML instance. 

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one. 

In [65]:
SERVICE_PROVIDER_NAME = "Watson Machine Learning V2"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [66]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

**Note:** You can bind more than one engine instance if needed by calling `wos_client.service_providers.add` method. Next, you can refer to particular service provider using `service_provider_id`.

In [67]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "production",
        credentials=WMLCredentialsCloud(
            apikey=CLOUD_API_KEY,      ## use `apikey=IAM_TOKEN` if using IAM_TOKEN to initiate client
            url=WML_CREDENTIALS["url"],
            instance_id=None
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider dba91dcc-8186-45fd-8743-d6234cb7d34a 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [68]:
wos_client.service_providers.show()

33b72f90-25aa-4c6c-8de5-df9ebfb493f4,active,Watson Machine Learning V2,watson_machine_learning,2021-08-11 14:59:51.080000+00:00,dba91dcc-8186-45fd-8743-d6234cb7d34a
None,active,AWS Machine Learning,amazon_sagemaker,2021-08-09 17:47:09.325000+00:00,e106658e-9cb0-4439-b85c-6057fffb3421
None,active,Azure Machine Learning,azure_machine_learning,2021-07-29 12:50:37.092000+00:00,ba1b62a0-2a80-414e-9177-bcf71d5a8909


In [69]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_space_id = WML_SPACE_ID).result['resources'][0]
asset_deployment_details

{'metadata': {'guid': '62f113b9-7583-42a8-aa5c-b1754959fe83',
  'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/62f113b9-7583-42a8-aa5c-b1754959fe83?space_id=cc293f72-5440-478f-a9cc-6c137784f747',
  'created_at': '2021-08-11T14:26:30.035Z',
  'modified_at': '2021-08-11T14:26:30.035Z'},
 'entity': {'name': 'Spark German Risk Deployment - Final',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/62f113b9-7583-42a8-aa5c-b1754959fe83/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [70]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,deployment_space_id=WML_SPACE_ID)
model_asset_details_from_deployment

{'metadata': {'guid': '62f113b9-7583-42a8-aa5c-b1754959fe83',
  'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/62f113b9-7583-42a8-aa5c-b1754959fe83?space_id=cc293f72-5440-478f-a9cc-6c137784f747',
  'created_at': '2021-08-11T14:26:30.035Z',
  'modified_at': '2021-08-11T14:26:30.035Z'},
 'entity': {'name': 'Spark German Risk Deployment - Final',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/62f113b9-7583-42a8-aa5c-b1754959fe83/predictions'},
  'asset': {'asset_id': 'a12ed743-c537-436f-ab3c-fef1554b88d8',
   'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/models/a12ed743-c537-436f-ab3c-fef1554b88d8?space_id=cc293f72-5440-478f-a9cc-6c137784f747&version=2020-06-12',
   'name': 'Spark German Risk Model - Final',
   'asset_type': 'model',
   'created_at': '2021-08-11T14:26:12.354Z',
   'modified_at': '2021-08-11T14:26:19.018Z'},
  'asset_properties': {'model_type': 'mllib_2.4',
   'runtime_environment': 'spark-2.4',
   'lab

## Subscriptions

### Remove existing credit risk subscriptions

This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [71]:
wos_client.subscriptions.show()

6de5e798b8b64cc5d09e91734a149197,DaveGermanCredit.2019.2.12.17.16.52.617,a08b0e60-6844-4c14-81cb-a3a6c9d199f9,9764f091ac3099d6c5182764c7b97f90,DaveGermanCredit.2019.2.12.17.16.52.617,ba1b62a0-2a80-414e-9177-bcf71d5a8909,active,2021-07-29 12:51:37.381000+00:00,62d30e9f-8fdc-4006-88f5-f76a7edb72e7


This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [72]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

### This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [73]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest

In [74]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['metadata']['url'],
            scoring_endpoint=ScoringEndpointRequest(url=scoring_url) # score model without shadow deployment
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='Risk',
            probability_fields=['probability'],
            prediction_field='predictedLabel',
            feature_fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
            categorical_fields = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
            training_data_reference=TrainingDataReference(type='cos',
                                                          location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                                    file_name = training_data_file_name),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": COS_RESOURCE_CRN,
                                                                        "url": COS_ENDPOINT,
                                                                        "api_key": COS_API_KEY_ID,
                                                                        "iam_url": IAM_URL})),
            training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
subscription_id

'c752d566-1517-44bc-9c5c-b0b2c1ab59c0'

In [75]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  d72f4bc9-6f53-4ad2-a0e6-1ca13b2f1139


In [76]:
wos_client.data_sets.show()

a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,c752d566-1517-44bc-9c5c-b0b2c1ab59c0,subscription,payload_logging_error,2021-08-11 15:00:18.776000+00:00,ddf4a9ad-d1f5-4655-91f0-603d2216f961
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,c752d566-1517-44bc-9c5c-b0b2c1ab59c0,subscription,manual_labeling,2021-08-11 15:00:18.470000+00:00,cc141027-57dd-498d-80b7-f8d6151e4cf1
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,c752d566-1517-44bc-9c5c-b0b2c1ab59c0,subscription,payload_logging,2021-08-11 15:00:18.106000+00:00,d72f4bc9-6f53-4ad2-a0e6-1ca13b2f1139
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,62d30e9f-8fdc-4006-88f5-f76a7edb72e7,subscription,payload_logging,2021-07-29 12:51:38.316000+00:00,eedaadcd-2c2d-4326-9278-25569d957852
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,62d30e9f-8fdc-4006-88f5-f76a7edb72e7,subscription,manual_labeling,2021-07-29 12:51:38.410000+00:00,6063f63a-c808-48c9-8ab0-ee188ede6545
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,62d30e9f-8fdc-4006-88f5-f76a7edb72e7,subscription,feedback,2021-07-29 13:04:35.424000+00:00,aeb83572-e853-41a0-8d4c-c2e92dc18d37
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,62d30e9f-8fdc-4006-88f5-f76a7edb72e7,subscription,payload_logging_error,2021-07-29 12:51:38.505000+00:00,86d19448-b0bc-433c-9c08-625133f82348
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,a08b0e60-6844-4c14-81cb-a3a6c9d199f9,data_mart,explanations,2021-07-28 17:15:20.374000+00:00,6a170500-45e0-41ac-8da7-5ac18b007d6c


Get subscription list

In [77]:
wos_client.subscriptions.show()

a12ed743-c537-436f-ab3c-fef1554b88d8,Spark German Risk Model - Final,a08b0e60-6844-4c14-81cb-a3a6c9d199f9,62f113b9-7583-42a8-aa5c-b1754959fe83,Spark German Risk Deployment - Final,dba91dcc-8186-45fd-8743-d6234cb7d34a,active,2021-08-11 15:00:16.406000+00:00,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
6de5e798b8b64cc5d09e91734a149197,DaveGermanCredit.2019.2.12.17.16.52.617,a08b0e60-6844-4c14-81cb-a3a6c9d199f9,9764f091ac3099d6c5182764c7b97f90,DaveGermanCredit.2019.2.12.17.16.52.617,ba1b62a0-2a80-414e-9177-bcf71d5a8909,active,2021-07-29 12:51:37.381000+00:00,62d30e9f-8fdc-4006-88f5-f76a7edb72e7


### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [78]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
predictions = wml_client.deployments.score(deployment_uid, payload_scoring)

print("Single record scoring result:", "\n fields:", predictions["predictions"][0]["fields"], "\n values: ", predictions["predictions"][0]["values"][0])

Single record scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'LoanPurpose_IX', 'ExistingSavings_IX', 'EmploymentDuration_IX', 'Sex_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'InstallmentPlans_IX', 'Housing_IX', 'Job_IX', 'Telephone_IX', 'ForeignWorker_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [20, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19], [1.0, 1.0, 1.0, 13.0, 

## Check if WML payload logging worked else manually store payload records

In [79]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": predictions['predictions'][0]['fields'], "values":predictions['predictions'][0]['values']},
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Number of records in the payload logging table: 8


# Quality monitoring and feedback logging <a name="quality"></a>

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [80]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation c8a6c276-f228-49d8-9b89-c0f4b8b3e231 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [81]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'c8a6c276-f228-49d8-9b89-c0f4b8b3e231'

## Feedback logging

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [82]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json

rm: cannot remove 'additional_feedback_data_v2.json': No such file or directory
--2021-08-11 15:02:10--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

additional_feedback 100%[===================>]  49.70K  --.-KB/s    in 0.001s  

2021-08-11 15:02:10 (54.6 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]



### Get feedback logging dataset ID

In [83]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

{
  "data_sets": [
    {
      "metadata": {
        "id": "2bf80f97-efd3-4d5e-99a6-d8f0b6428ba7",
        "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/181ed6cc388f47bd9d862fe066f9cfce:a08b0e60-6844-4c14-81cb-a3a6c9d199f9:data_set:2bf80f97-efd3-4d5e-99a6-d8f0b6428ba7",
        "url": "/v2/data_sets/2bf80f97-efd3-4d5e-99a6-d8f0b6428ba7",
        "created_at": "2021-08-11T15:00:58.806000Z",
        "created_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad",
        "modified_at": "2021-08-11T15:00:59.309000Z",
        "modified_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad"
      },
      "entity": {
        "data_mart_id": "a08b0e60-6844-4c14-81cb-a3a6c9d199f9",
        "name": "c752d566-1517-44bc-9c5c-b0b2c1ab59c0_feedback",
        "description": "c752d566-1517-44bc-9c5c-b0b2c1ab59c0_feedback",
        "type": "feedback",
        "target": {
          "target_type": "subscription",
          "target_id": "c752d566-1517-44bc-9c5c-b0b2c1ab59c0"
        },
    

In [84]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

In [85]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)




 Waiting for end of storing records with request id: d87f7791-47af-4b44-9438-d535bba601ed 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [86]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

98

In [87]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1 




running...
finished

---------------------------
 Successfully finished run 
---------------------------




In [88]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2021-08-11 15:02:30.493000+00:00,true_positive_rate,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.36363636363636365,None,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:02:30.493000+00:00,area_under_roc,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.6510489510489511,0.8,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:02:30.493000+00:00,precision,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.75,None,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:02:30.493000+00:00,f1_measure,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.48979591836734687,None,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:02:30.493000+00:00,accuracy,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.7448979591836735,None,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:02:30.493000+00:00,log_loss,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.4616535286562782,None,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:02:30.493000+00:00,false_positive_rate,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.06153846153846154,None,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:02:30.493000+00:00,area_under_pr,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.6185064935064934,None,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:02:30.493000+00:00,recall,5cd34d4a-13c6-44d5-ba26-fcec9624ec86,0.36363636363636365,None,None,['model_type:original'],quality,c8a6c276-f228-49d8-9b89-c0f4b8b3e231,dacfdeb5-9d8f-4a75-9c24-68f0ec7ab0d1,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0


# Fairness, drift monitoring and explanations <a name="fairness"></a>

### Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:

  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [89]:
wos_client.monitor_instances.show()

a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,c752d566-1517-44bc-9c5c-b0b2c1ab59c0,subscription,quality,2021-08-11 15:00:58.147000+00:00,c8a6c276-f228-49d8-9b89-c0f4b8b3e231
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,62d30e9f-8fdc-4006-88f5-f76a7edb72e7,subscription,fairness,2021-07-29 13:05:05.409000+00:00,3a68d116-dc8f-4fd2-aba1-9e10dabb3797
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,62d30e9f-8fdc-4006-88f5-f76a7edb72e7,subscription,drift,2021-07-29 13:05:59.474000+00:00,d7d1cdc6-ae66-44c5-9c87-1464fc00a60e
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,62d30e9f-8fdc-4006-88f5-f76a7edb72e7,subscription,explainability,2021-07-29 13:05:49.378000+00:00,fb780969-54eb-4b50-9687-bcbb4d2712cd
a08b0e60-6844-4c14-81cb-a3a6c9d199f9,active,62d30e9f-8fdc-4006-88f5-f76a7edb72e7,subscription,quality,2021-07-29 13:04:34.663000+00:00,4a675973-a9ce-4f7a-891e-0d458f5a08b3


In [90]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id




 Waiting for end of monitor instance creation 5dbe7ec5-2307-4ef1-a248-d7c63cec4414 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'5dbe7ec5-2307-4ef1-a248-d7c63cec4414'

### Drift configuration

####  Note: you can choose to enable/disable (True or False) model or data drift within config

In [91]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 100,
    "drift_threshold": 0.1,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation e98e4263-be05-4b60-bf0c-314ac4d21c5b 




preparing.
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'e98e4263-be05-4b60-bf0c-314ac4d21c5b'

## Score the model again now that monitoring is configured

This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness.

In [92]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_feed.json
!ls -lh german_credit_feed.json

-rw-rw---- 1 wsuser watsonstudio 3.0M Aug 11 15:03 german_credit_feed.json


Score 200 randomly chosen records

In [93]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

scoring_response = wml_client.deployments.score(deployment_uid, payload_scoring)

time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)

if pl_records_count == 8:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": scoring_response['predictions'][0]['fields'], "values":scoring_response['predictions'][0]['values']},
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

**Note:** Now in payload table should be total 208 records.

In [94]:
print('Number of records in payload table: ', wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id))

Number of records in payload table:  208


## Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface.

In [95]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 43c2666e-f7d8-4f2b-81a2-5ed527713fcf 




running..
finished

---------------------------
 Successfully finished run 
---------------------------




In [96]:
time.sleep(10)

wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2021-08-11 15:03:58.051795+00:00,fairness_value,aeba22aa-1c97-4a56-932b-a38af4bacf13,86.8,80.0,None,"['feature:Sex', 'fairness_metric_type:debiased_fairness', 'feature_value:female']",fairness,5dbe7ec5-2307-4ef1-a248-d7c63cec4414,43c2666e-f7d8-4f2b-81a2-5ed527713fcf,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:03:58.051795+00:00,fairness_value,aeba22aa-1c97-4a56-932b-a38af4bacf13,112.00000000000001,80.0,None,"['feature:Age', 'fairness_metric_type:debiased_fairness', 'feature_value:18-25']",fairness,5dbe7ec5-2307-4ef1-a248-d7c63cec4414,43c2666e-f7d8-4f2b-81a2-5ed527713fcf,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:03:58.051795+00:00,fairness_value,38d7293b-2df5-47e2-b25d-3812bc4f4be4,86.8,80.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,5dbe7ec5-2307-4ef1-a248-d7c63cec4414,43c2666e-f7d8-4f2b-81a2-5ed527713fcf,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0
2021-08-11 15:03:58.051795+00:00,fairness_value,38d7293b-2df5-47e2-b25d-3812bc4f4be4,112.00000000000001,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,5dbe7ec5-2307-4ef1-a248-d7c63cec4414,43c2666e-f7d8-4f2b-81a2-5ed527713fcf,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0


## Run drift monitor


Kick off a drift monitor run on current data. The monitor runs every hour, but can be manually initiated using the Python client, the REST API.

In [97]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 68ddbc52-5818-4921-88bf-2b05f9fffa58 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [98]:
time.sleep(5)

wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2021-08-11 15:04:33.716013+00:00,data_drift_magnitude,50a5f766-b921-497f-9ab3-1a52d1913045,0.08173076923076923,None,None,[],drift,e98e4263-be05-4b60-bf0c-314ac4d21c5b,68ddbc52-5818-4921-88bf-2b05f9fffa58,subscription,c752d566-1517-44bc-9c5c-b0b2c1ab59c0


## Configure Explainability

Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [99]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation 8eaf3ceb-67be-4b7f-95c5-ed595c574c36 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Run explanation for sample record

In [100]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)

Running explanations on scoring IDs: ['d0e532b2e6bf3154d5880419eb7bad74-1']
{
  "metadata": {
    "explanation_task_ids": [
      "88eafc4f-5b29-4fa5-a7a5-0213857a08c5"
    ],
    "created_by": "IBMid-50M71Q0T71",
    "created_at": "2021-08-11T15:05:01.282102Z"
  }
}


# Custom monitors and metrics <a name="custom"></a>

## Register custom monitor

In [101]:
def get_definition(monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    
    for definition in monitor_definitions:
        if monitor_name == definition.entity.name:
            return definition
    
    return None

In [102]:
monitor_name = 'my model performance'
metrics = [MonitorMetricRequest(name='sensitivity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
          MonitorMetricRequest(name='specificity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.75)])]
tags = [MonitorTagRequest(name='region', description='customer geographical region')]

existing_definition = get_definition(monitor_name)

if existing_definition is None:
    custom_monitor_details = wos_client.monitor_definitions.add(name=monitor_name, metrics=metrics, tags=tags, background_mode=False).result
else:
    custom_monitor_details = existing_definition




 Waiting for end of adding monitor definition my_model_performance 




finished

-------------------------------------------------
 Successfully finished adding monitor definition 
-------------------------------------------------




### Show available monitors types

In [103]:
wos_client.monitor_definitions.show()

my_model_performance,my model performance,"['sensitivity', 'specificity']"
assurance,Assurance,"['Uncertainty', 'Confidence']"
fairness,Fairness,"['Fairness value', 'Average Odds Difference metric value', 'False Discovery Rate Difference metric value', 'Error Rate Difference metric value', 'False Negative Rate Difference metric value', 'False Omission Rate Difference metric value', 'False Positive Rate Difference metric value', 'True Positive Rate Difference metric value']"
performance,Performance,['Number of records']
explainability,Explainability,[]
mrm,Model risk management monitoring,"['Tests run', 'Tests passed', 'Tests failed', 'Tests skipped', 'Fairness score', 'Quality score', 'Drift score']"
correlations,Correlations,"['Maximum positive correlation coefficient', 'Maximum negative correlation coefficient', 'Mean absolute correlation coefficient', 'Significant correlation coefficients count']"
drift,Drift,"['Drop in accuracy', 'Predicted accuracy', 'Drop in data consistency']"
quality,Quality,"['Area under ROC', 'Area under PR', 'Proportion explained variance', 'Mean absolute error', 'Mean squared error', 'R squared', 'Root of mean squared error', 'Accuracy', 'Weighted True Positive Rate (wTPR)', 'True positive rate (TPR)', 'Weighted False Positive Rate (wFPR)', 'False positive rate (FPR)', 'Weighted recall', 'Recall', 'Weighted precision', 'Precision', 'Weighted F1-Measure', 'F1-Measure', 'Logarithmic loss']"


### Get monitors uids and details

In [104]:
custom_monitor_id = custom_monitor_details.metadata.id

print(custom_monitor_id)

my_model_performance


In [105]:
custom_monitor_details = wos_client.monitor_definitions.get(monitor_definition_id=custom_monitor_id).result
print('Monitor definition details:', custom_monitor_details)

Monitor definition details: {
  "metadata": {
    "id": "my_model_performance",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/46816354d9cb773bae86c226aa74c8cc:a08b0e60-6844-4c14-81cb-a3a6c9d199f9:monitor_definition:my_model_performance",
    "url": "/v2/monitor_definitions/my_model_performance",
    "created_at": "2021-08-11T15:05:07.026000Z",
    "created_by": "IBMid-50M71Q0T71"
  },
  "entity": {
    "name": "my model performance",
    "metrics": [
      {
        "name": "sensitivity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.8
          }
        ],
        "expected_direction": "increasing",
        "id": "sensitivity"
      },
      {
        "name": "specificity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.75
          }
        ],
        "expected_direction": "increasing",
        "id": "specificity"
      }
    ],
    "tags": [
      {
        "name": "region

## Enable custom monitor for subscription

In [106]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
    )

thresholds = [MetricThresholdOverride(metric_id='sensitivity', type = MetricThresholdTypes.LOWER_LIMIT, value=0.9)]

custom_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=custom_monitor_id,
            target=target
).result




 Waiting for end of monitor instance creation 643b2b0c-83d7-4c5c-94ca-24049d2432eb 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Get monitor instance id and configuration details

In [107]:
custom_monitor_instance_id = custom_monitor_instance_details.metadata.id

In [108]:
custom_monitor_instance_details = wos_client.monitor_instances.get(custom_monitor_instance_id).result
print(custom_monitor_instance_details)

{
  "metadata": {
    "id": "643b2b0c-83d7-4c5c-94ca-24049d2432eb",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/46816354d9cb773bae86c226aa74c8cc:a08b0e60-6844-4c14-81cb-a3a6c9d199f9:monitor_instance:643b2b0c-83d7-4c5c-94ca-24049d2432eb",
    "url": "/v2/monitor_instances/643b2b0c-83d7-4c5c-94ca-24049d2432eb",
    "created_at": "2021-08-11T15:05:24.158000Z",
    "created_by": "IBMid-50M71Q0T71"
  },
  "entity": {
    "data_mart_id": "a08b0e60-6844-4c14-81cb-a3a6c9d199f9",
    "monitor_definition_id": "my_model_performance",
    "target": {
      "target_type": "subscription",
      "target_id": "c752d566-1517-44bc-9c5c-b0b2c1ab59c0"
    },
    "thresholds": [
      {
        "metric_id": "sensitivity",
        "type": "lower_limit",
        "value": 0.8
      },
      {
        "metric_id": "specificity",
        "type": "lower_limit",
        "value": 0.75
      }
    ],
    "schedule": {
      "repeat_interval": 60,
      "repeat_unit": "minute",
      "repeat_type": "min

## Storing custom metrics

In [109]:
from datetime import datetime, timezone, timedelta
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
custom_monitoring_run_id = "11122223333111abc"
measurement_request = [MonitorMeasurementRequest(timestamp=datetime.now(timezone.utc), 
                                                 metrics=[{"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}], run_id=custom_monitoring_run_id)]
print(measurement_request[0])

{
  "timestamp": "2021-08-11T15:05:36.781958Z",
  "run_id": "11122223333111abc",
  "metrics": [
    {
      "specificity": 0.78,
      "sensitivity": 0.67,
      "region": "us-south"
    }
  ]
}


In [110]:
published_measurement_response = wos_client.monitor_instances.measurements.add(
    monitor_instance_id=custom_monitor_instance_id,
    monitor_measurement_request=measurement_request).result
published_measurement_id = published_measurement_response[0]["measurement_id"]
print(published_measurement_response)

[{'measurement_id': 'e769c3b8-9f36-46e5-bb65-03d3c158171d', 'metrics': [{'region': 'us-south', 'sensitivity': 0.67, 'specificity': 0.78}], 'run_id': '11122223333111abc', 'timestamp': '2021-08-11T15:05:36.781958Z'}]


### List and get custom metrics

In [111]:
time.sleep(5)
published_measurement = wos_client.monitor_instances.measurements.get(monitor_instance_id=custom_monitor_instance_id, measurement_id=published_measurement_id).result
print(published_measurement)

{
  "metadata": {
    "id": "e769c3b8-9f36-46e5-bb65-03d3c158171d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/46816354d9cb773bae86c226aa74c8cc:a08b0e60-6844-4c14-81cb-a3a6c9d199f9:measurement:e769c3b8-9f36-46e5-bb65-03d3c158171d",
    "url": "/v2/monitor_instances/643b2b0c-83d7-4c5c-94ca-24049d2432eb/measurements/e769c3b8-9f36-46e5-bb65-03d3c158171d",
    "created_at": "2021-08-11T15:05:39.663000Z",
    "created_by": "IBMid-50M71Q0T71"
  },
  "entity": {
    "timestamp": "2021-08-11T15:05:36.781958Z",
    "run_id": "11122223333111abc",
    "values": [
      {
        "metrics": [
          {
            "id": "sensitivity",
            "value": 0.67,
            "lower_limit": 0.8
          },
          {
            "id": "specificity",
            "value": 0.78,
            "lower_limit": 0.75
          }
        ],
        "tags": [
          {
            "id": "region",
            "value": "us-south"
          }
        ]
      }
    ],
    "issue_count": 1,
    "t

# Historical data <a name="historical"></a>

In [112]:
historyDays = 7

## Insert historical fairness metrics

In [113]:
!rm history_fairness_v2.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_fairness_v2.json
!ls -lh history_fairness_v2.json

rm: cannot remove 'history_fairness_v2.json': No such file or directory
-rw-rw---- 1 wsuser watsonstudio 37K Aug 11 15:05 history_fairness_v2.json


In [114]:
from datetime import datetime, timedelta, timezone

with open('history_fairness_v2.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    daily_measurement_requests = []
    
    for hour in range(24):
        score_time = datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed
 
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [payloads[index][0], payloads[index][1]])
        daily_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=fairness_monitor_instance_id,
                                            monitor_measurement_request=daily_measurement_requests).result     
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical debias metrics

In [115]:
!rm history_debias_v2.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_debias_v2.json
!ls -lh history_debias_v2.json

rm: cannot remove 'history_debias_v2.json': No such file or directory
-rw-rw---- 1 wsuser watsonstudio 37K Aug 11 15:06 history_debias_v2.json


In [116]:
with open('history_debias_v2.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    daily_measurement_requests = []
    for hour in range(24):
        score_time = datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed

        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [payloads[index][0], payloads[index][1]])
        
        daily_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=fairness_monitor_instance_id,
                                            monitor_measurement_request=daily_measurement_requests).result     

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical quality metrics

In [117]:
measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    quality_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        
        metric = {"area_under_roc": measurements[day]}
                
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [metric])
        quality_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=quality_monitor_instance_id,
                                            monitor_measurement_request=quality_measurement_requests).result    
    
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical confusion matrixes

In [118]:
!rm history_quality_metrics.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_quality_metrics.json
!ls -lh history_quality_metrics.json

rm: cannot remove 'history_quality_metrics.json': No such file or directory
-rw-rw---- 1 wsuser watsonstudio 79K Aug 11 15:06 history_quality_metrics.json


In [119]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source

with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)
    
for day in range(historyDays):
    index = 0
    cm_measurement_requests = []
    print('Loading day', day + 1)
    
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"

        metric = records[index]['metrics']
        source = records[index]['sources']

        
        measurement_request = {"timestamp": score_time, "metrics": [metric], "sources": [source]}
        cm_measurement_requests.append(measurement_request)

        index+=1

    response = wos_client.monitor_instances.measurements.add(monitor_instance_id=quality_monitor_instance_id, monitor_measurement_request=cm_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical performance metrics

In [120]:
target = Target(
        target_type=TargetTypes.INSTANCE,
        target_id=payload_data_set_id
    )


performance_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=wos_client.monitor_definitions.MONITORS.PERFORMANCE.ID,
            target=target
).result
performance_monitor_instance_id = performance_monitor_instance_details.metadata.id





 Waiting for end of monitor instance creation 2d58c8e1-bf6c-4c10-93d2-88e6221ad6fd 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [121]:
for day in range(historyDays):
    performance_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        score_count = random.randint(60, 600)
        
        metric = {"record_count": score_count, "data_set_type": "scoring_payload"}
        
        measurement_request = {"timestamp": score_time, "metrics": [metric]}
        performance_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=performance_monitor_instance_id,
                                            monitor_measurement_request=performance_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical drift measurements

In [122]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_0.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_1.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_2.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_3.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_4.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_5.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_6.json
!ls -lh history_drift_measurement_*.json

-rw-rw---- 1 wsuser watsonstudio 832K Aug 11 15:07 history_drift_measurement_0.json
-rw-rw---- 1 wsuser watsonstudio 868K Aug 11 15:07 history_drift_measurement_1.json
-rw-rw---- 1 wsuser watsonstudio 870K Aug 11 15:07 history_drift_measurement_2.json
-rw-rw---- 1 wsuser watsonstudio 910K Aug 11 15:07 history_drift_measurement_3.json
-rw-rw---- 1 wsuser watsonstudio 841K Aug 11 15:07 history_drift_measurement_4.json
-rw-rw---- 1 wsuser watsonstudio 836K Aug 11 15:07 history_drift_measurement_5.json
-rw-rw---- 1 wsuser watsonstudio 840K Aug 11 15:07 history_drift_measurement_6.json


In [123]:
for day in range(historyDays):
    drift_measurements = []

    with open("history_drift_measurement_{}.json".format(day), 'r') as history_file:
        drift_daily_measurements = json.load(history_file)
    print('Loading day', day + 1)

    #Historical data contains 8 records per day - each represents 3 hour drift window.
    
    for nb_window, records in enumerate(drift_daily_measurements):
        for record in records:
            window_start =  datetime.utcnow() + timedelta(hours=(-(24 * day + (nb_window+1)*3 + 1))) # first_payload_record_timestamp_in_window (oldest)
            window_end = datetime.utcnow() + timedelta(hours=(-(24 * day + nb_window*3 + 1)))# last_payload_record_timestamp_in_window (most recent)
            #modify start and end time for each record
            record['sources'][0]['data']['start'] = window_start.isoformat() + "Z"
            record['sources'][0]['data']['end'] = window_end.isoformat() + "Z"
            
            
            metric = record['metrics'][0]
            source = record['sources'][0]

            measurement_request = {"timestamp": window_start.isoformat() + "Z", "metrics": [metric], "sources": [source]}
            
            drift_measurements.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=drift_monitor_instance_id,
                                            monitor_measurement_request=drift_measurements).result    

    
    print("Daily loading finished.")

Loading day 1
Daily loading finished.
Loading day 2
Daily loading finished.
Loading day 3
Daily loading finished.
Loading day 4
Daily loading finished.
Loading day 5
Daily loading finished.
Loading day 6
Daily loading finished.
Loading day 7
Daily loading finished.


## Additional data to help debugging

In [124]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)

Datamart: a08b0e60-6844-4c14-81cb-a3a6c9d199f9
Model: a12ed743-c537-436f-ab3c-fef1554b88d8
Deployment: 62f113b9-7583-42a8-aa5c-b1754959fe83


## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [125]:
wos_client.data_sets.show_records(payload_data_set_id, limit=5)

,"[0.710052640247997, 0.28994735975200303]",4450,d0e532b2e6bf3154d5880419eb7bad74-1,1.0,7.0,0.0,3,100_to_500,2021-08-11T15:03:44.358Z,0.710052640247997,None,"[20, [2, 3, 4, 5, 7, 13, 14, 15, 16, 17, 18, 19], [7.0, 1.0, 3.0, 1.0, 2.0, 28.0, 4450.0, 3.0, 4.0, 24.0, 1.0, 1.0]]",no_checking,education,No Risk,2.0,0.0,skilled,none,1.0,0.0,none,28,24,prior_payments_delayed,0.0,4,0.0,3.0,0.0,0.0,yes,own,0.0,1,62f113b9-7583-42a8-aa5c-b1754959fe83,"[14.201052804959941, 5.7989471950400615]",none,0.0,female,1,real_estate,less_1,No Risk,"[0.710052640247997, 0.28994735975200303]"
,"[0.3752783644038116, 0.6247216355961884]",6311,d0e532b2e6bf3154d5880419eb7bad74-10,0.0,1.0,0.0,3,greater_1000,2021-08-11T15:03:44.358Z,0.6247216355961884,None,"[0.0, 2.0, 1.0, 3.0, 1.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 32.0, 6311.0, 3.0, 5.0, 32.0, 2.0, 1.0]",no_checking,furniture,Risk,3.0,1.0,skilled,none,3.0,2.0,co-applicant,32,32,outstanding_credit,0.0,5,0.0,1.0,0.0,0.0,yes,own,0.0,2,62f113b9-7583-42a8-aa5c-b1754959fe83,"[7.505567288076232, 12.494432711923768]",none,1.0,male,1,unknown,4_to_7,Risk,"[0.3752783644038116, 0.6247216355961884]"
,"[0.9469841759655155, 0.053015824034484384]",250,d0e532b2e6bf3154d5880419eb7bad74-100,0.0,2.0,1.0,1,less_100,2021-08-11T15:03:44.358Z,0.9469841759655155,None,"[1.0, 3.0, 2.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 250.0, 1.0, 2.0, 19.0, 1.0, 1.0]",less_0,car_used,No Risk,2.0,0.0,skilled,none,0.0,3.0,none,4,19,all_credits_paid_back,0.0,2,0.0,4.0,0.0,0.0,yes,rent,1.0,1,62f113b9-7583-42a8-aa5c-b1754959fe83,"[18.93968351931031, 1.0603164806896876]",none,0.0,male,1,real_estate,unemployed,No Risk,"[0.9469841759655155, 0.053015824034484384]"
,"[0.6562365934739304, 0.34376340652606974]",4879,d0e532b2e6bf3154d5880419eb7bad74-101,0.0,3.0,0.0,4,greater_1000,2021-08-11T15:03:44.358Z,0.6562365934739304,None,"[20, [1, 2, 3, 4, 7, 13, 14, 15, 16, 17, 18, 19], [1.0, 3.0, 3.0, 2.0, 3.0, 17.0, 4879.0, 4.0, 4.0, 38.0, 2.0, 1.0]]",no_checking,radio_tv,No Risk,3.0,0.0,skilled,none,3.0,1.0,none,17,38,credits_paid_to_date,0.0,4,0.0,2.0,0.0,0.0,yes,own,0.0,2,62f113b9-7583-42a8-aa5c-b1754959fe83,"[13.124731869478607, 6.875268130521395]",none,0.0,male,1,unknown,greater_7,No Risk,"[0.6562365934739304, 0.34376340652606974]"
,"[0.9235957628842083, 0.07640423711579161]",452,d0e532b2e6bf3154d5880419eb7bad74-102,1.0,0.0,1.0,2,less_100,2021-08-11T15:03:44.358Z,0.9235957628842083,None,"[20, [0, 1, 4, 5, 9, 13, 14, 15, 16, 17, 18, 19], [1.0, 3.0, 3.0, 1.0, 1.0, 8.0, 452.0, 2.0, 2.0, 19.0, 1.0, 1.0]]",less_0,car_new,No Risk,0.0,0.0,skilled,none,0.0,3.0,none,8,19,all_credits_paid_back,0.0,2,0.0,3.0,0.0,0.0,yes,rent,1.0,1,62f113b9-7583-42a8-aa5c-b1754959fe83,"[18.471915257684167, 1.5280847423158321]",none,0.0,female,1,savings_insurance,less_1,No Risk,"[0.9235957628842083, 0.07640423711579161]"


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

## Next steps

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary.